In [5]:
import numpy as np
import pandas as pd
import cv2
import os
import tensorflow as tf


In [8]:
zip_path = "C:/Users/Admin/Downloads/Telegram Desktop/blurred 25.zip"

In [10]:
from zipfile import ZipFile


In [ ]:
with ZipFile(zip_path, 'r') as zip:
    zip.extractall('folder')
    print('Done!')

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Reshape, Conv2D, Conv2DTranspose, LeakyReLU, BatchNormalization, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Enable GPU acceleration (if available)
print("GPU Available:", tf.config.list_physical_devices('GPU'))


In [13]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [15]:
IMG_SIZE = 128
BATCH_SIZE = 32
dataset_path = "C:/Users/Admin/Downloads/Telegram Desktop/blurred 25" 

In [ ]:
# Load and resize images
def load_images(path, img_size=IMG_SIZE):
    images = []
    filenames = os.listdir(path)[:1001]  # Limit to 1001 images
    for file in filenames:
        img = load_img(os.path.join(path, file), target_size=(img_size, img_size), color_mode="grayscale")
        img = img_to_array(img) / 255.0  # Normalize to [0,1]
        images.append(img)
    return np.array(images)

# Load dataset
X_train = load_images(dataset_path)
print(f"Loaded {X_train.shape[0]} images with shape {X_train.shape[1:]}")

In [ ]:
datagen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)

def augment_images(images):
    return next(datagen.flow(images, batch_size=len(images), shuffle=False))

# Apply augmentation to dataset
X_train = augment_images(X_train)
print("Augmented images shape:", X_train.shape)


In [ ]:
def build_generator(latent_dim):
    model = Sequential([
        Dense(8 * 8 * 256, activation="relu", input_dim=latent_dim),
        Reshape((8, 8, 256)),

        Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
        LeakyReLU(alpha=0.2),

        Conv2DTranspose(64, kernel_size=4, strides=2, padding="same"),
        LeakyReLU(alpha=0.2),

        Conv2DTranspose(32, kernel_size=4, strides=2, padding="same"),
        LeakyReLU(alpha=0.2),

        Conv2DTranspose(1, kernel_size=4, strides=2, padding="same", activation="tanh")  # Now 128x128x1
    ])
    return model

generator = build_generator(100)


In [ ]:
def build_discriminator(img_shape=(128, 128, 1)):  # Ensure shape matches your images
    model = Sequential([
        Conv2D(64, kernel_size=3, strides=2, input_shape=img_shape, padding="same"),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        
        Conv2D(128, kernel_size=3, strides=2, padding="same"),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        
        Flatten(),  # Flatten into a 1D vector
        Dense(1, activation="sigmoid")  # Binary classification
    ])
    
    model.compile(loss="binary_crossentropy", optimizer=Adam(0.0002, 0.5), metrics=["accuracy"])
    return model

discriminator = build_discriminator()


In [20]:
def build_gan(generator, discriminator):
    discriminator.trainable = False
    gan_input = Input(shape=(100,))
    img = generator(gan_input)
    validity = discriminator(img)
    model = Model(gan_input, validity)
    model.compile(loss="binary_crossentropy", optimizer=Adam(0.0002, 0.5))
    return model

In [21]:
gan = build_gan(generator, discriminator)


In [22]:
import time

EPOCHS = 3000
BATCH_SIZE = 32
SAVE_INTERVAL = 500
LATENT_DIM = 100

real_labels = np.ones((BATCH_SIZE, 1))
fake_labels = np.zeros((BATCH_SIZE, 1))

start_time = time.time()

In [ ]:
for epoch in range(1, 1001):  # Reduce epochs if needed
    idx = np.random.randint(0, X_train.shape[0], BATCH_SIZE)
    real_images = X_train[idx]

    noise = np.random.normal(0, 1, (BATCH_SIZE, 100))
    generated_images = generator.predict(noise)

    d_loss_real = discriminator.train_on_batch(real_images, real_labels)
    d_loss_fake = discriminator.train_on_batch(generated_images, fake_labels)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    noise = np.random.normal(0, 1, (BATCH_SIZE, 100))
    g_loss = gan.train_on_batch(noise, real_labels)

    if epoch % 100 == 0:  
        generator.save(f"generator_epoch_{epoch}.h5")

    if epoch % 500 == 0:
        print(f"Epoch {epoch} | D Loss: {d_loss[0]:.4f} | G Loss: {g_loss:.4f}")


In [ ]:
def generate_images(generator, n_images=5):
    noise = np.random.normal(0, 1, (n_images, 100))
    generated_images = generator.predict(noise)

    plt.figure(figsize=(10, 5))
    for i in range(n_images):
        plt.subplot(1, n_images, i + 1)
        plt.imshow(generated_images[i, :, :, 0], cmap="gray")
        plt.axis("off")
    plt.show()

# Generate and display 5 fake X-rays
generate_images(generator, 5)
